# Packages

In [1]:
import sys
import utils as ut
from utils import flush
import json
import re

# Input

In [2]:
def L2(f):
    string = f.readline()
    JOBNAM = string[10:20].strip()
    MSGL =  int(string[27:30])
    return JOBNAM, MSGL
def L3(f):
    string = f.readline()
    FOR001 = string[7:].strip()
    return FOR001
def L6(f):
    string = f.readline()
    Lmax = int(string[7:11])
    NSR = int(string[17:21])
    NFI = int(string[27:30])
    return Lmax, NSR, NFI
def L7(f):
    string = f.readline()
    NPRN = int(string[7:11])
    IVEF = int(string[17:21])
    return NPRN, IVEF
    

In [3]:
filename="lat_bcc.dat"
with open(filename) as f:
    flush(f)
    JOBNAM, MSGL = L2(f)
    # print(JOBNAM, MSGL)
    FOR001 = L3(f)
    # print(FOR001)
    flush(f,2)
    Lmax, NSR, NFI = L6(f)
    NPRN, IVEF = L7(f)
    # print(Lmax, NSR, NFI, NPRN, IVEF)
    
# Combine entities
Input = {
    'Meta':{
        'JOBNAM': [JOBNAM, None],
        'MSGL': [MSGL, None],
        'FOR001': [FOR001, None],
        'NPRN': [NPRN, None],
    },
    'Approximation':{
        'Lmax': [Lmax, None],
        'NSR': [NSR, None],
        'NFI': [NFI, None],
        'IVEF': [IVEF, None]
    }
}
    

In [4]:
Input

{'Meta': {'JOBNAM': ['lat_bcc', None],
  'MSGL': [1, None],
  'FOR001': ['../../kstr/lat_bcc/smx/lat_bcc.tfh', None],
  'NPRN': [0, None]},
 'Approximation': {'Lmax': [30, None],
  'NSR': [129, None],
  'NFI': [11, None],
  'IVEF': [3, None]}}

### Test new method

In [5]:

data = {}
with open("lat_bcc.dat") as f:
    lines = f.readlines()

    for line in lines:
        match = re.search(r"(\w+)\.=([\w\/\.]+)", line)
        if match:
            key, value = match.groups()
            try:
                # Try to convert numerical values to integers
                value = int(value)
            except ValueError:
                try:
                    # Try to convert numerical values to floats
                    value = float(value)
                except ValueError:
                    pass  # If not a number, keep the value as string
            data[key] = value

with open('database.json', 'w') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)


# Output

In [6]:
def FHNDLR1(f):
    # read time
    timestamp = ut.head_timestamp(f)
    return timestamp
def FHNDLR2(f):
    # read FOR001
    FOR001 = ut.css(f)
    # flush 2 lines (FOR002, FOOR006)
    flush(f,2)
    return FOR001
def FHNDLR3(f):
    # read system information
    EMTO = ut.css(f)
    branch = ut.css(f)
    hash_key = ut.css(f)
    compile_on = ut.css(f)
    OS = ut.css(f)
    CPU = ut.css(f)
    compiler = ut.css(f)
    library = ut.css(f)
    return EMTO, branch, hash_key, compile_on, OS, CPU, compiler, library
def INPUT(f):
    # NSR, LMAX, NFI, IVEF
    NSR, LMAX, NFI, IVEF = ut.e4i(f)
    return NSR, LMAX, NFI, IVEF
def TRNSFM(f):
    # slope_matrices, KW2
    string = f.readline()
    slope_matrices = string[26:36].strip()
    KW2 = int(string[50:60])
    return slope_matrices, KW2
def BLATTS(f):
    # format {plane_num: 1, plane_info:[x,y,z,d], point_num: 6, point_info:[[x,y,z,d(상대값)],...]}
    
    
    
def SITES_INFO(f):
    # site num
    try:
        site_num_line = f.readline()
    except:
        raise ValueError("No more site information")
        return None
    if "Site number" not in site_num_line:
        raise ValueError("No more site information")
        return None
    token = site_num_line.split(': ')
    site_num = int(token[1].strip())
    
    # BLATTS
    Si, WFA, SC = ut.e3f(f)
    NSC, NVN = ut.e2f(f)
    
    


IndentationError: expected an indented block (<ipython-input-6-47b8238f0064>, line 37)

In [22]:
filename="lat_bcc.prn"
### 빈줄 제거
with open(filename, 'r') as f:
    lines = f.readlines()
lines = [line for line in lines if line.strip()]
with open('temp_'+filename, 'w') as f:
    f.writelines(lines)
    
### 데이터 읽기
with open('temp_'+filename, 'r') as f:
    # FHNDLR
    time_start = FHNDLR1(f)
    FOR001 = FHNDLR2(f)
    EMTO, branch, hash_key, compile_on, OS, CPU, compiler, library = FHNDLR3(f)
        
    print(time_start, FOR001, EMTO, branch, hash_key, compile_on, OS, CPU, compiler, library)
    
    # INPUT
    
    # TRNSFM
    
    # BLATTS
    
    # RSORT
    
    # RMESH
    
    # SETROT
    
    # UPDATE
    


1638473640.0 ../../kstr/lat_bcc/smx/lat_bcc.tfh not available not available not available Thu Apr  9 21:06:12 KST 2020 Linux 3.10.0-957.el7.x86_64 x86_64 GNU/Linux Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz ifort (IFORT) 18.0.5 20180823 mkl
